In [1]:
import pandas as pd
from pathlib import Path

In [2]:
def load_csvs_from_dir(directory, pattern="*.csv", concat=True, add_filename_col=True, sort=True, **read_csv_kwargs):
    """
    Load CSV files from `directory`.

    Args:
    - directory (str or Path): path to folder containing CSV files.
    - pattern (str): glob pattern to match files (default "*.csv").
    - concat (bool): if True, return a single concatenated DataFrame; if False, return a list of DataFrames.
    - add_filename_col (bool): if True, add a column '_source_file' with the filename for each row.
    - sort (bool): if True, sort files by name before loading.
    - **read_csv_kwargs: passed to `pd.read_csv` (e.g., parse_dates=["time"]).

    Returns:
    - pandas.DataFrame or list[pandas.DataFrame]
    """
    p = Path(directory)
    files = list(p.glob(pattern))
    if sort:
        files = sorted(files)
    dfs = []
    for f in files:
        try:
            df = pd.read_csv(f, **read_csv_kwargs)
        except Exception as e:
            # re-raise with context
            raise RuntimeError(f"Failed to read {f}: {e}") from e
        if add_filename_col:
            df["_source_file"] = f.name
        dfs.append(df)
    if concat:
        return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
    else:
        return dfs

In [18]:
powerwall_data_path="../tesla-solar-download/download/2733356/energy"
df_raw = load_csvs_from_dir(powerwall_data_path, add_filename_col=False)

In [19]:
df_raw.count()

timestamp                                93944
solar_energy_exported                    93944
grid_energy_imported                     93944
grid_energy_exported_from_solar          93944
grid_energy_exported_from_battery        93944
battery_energy_exported                  93944
battery_energy_imported_from_grid        93944
battery_energy_imported_from_solar       93944
consumer_energy_imported_from_grid       93944
consumer_energy_imported_from_solar      93944
consumer_energy_imported_from_battery    93944
raw_timestamp                            93944
total_battery_charge                         7
total_grid_energy_exported               18170
total_home_usage                         93916
total_battery_discharge                  34418
dtype: int64

In [20]:
df_raw.head()

,timestamp,solar_energy_exported,grid_energy_imported,grid_energy_exported_from_solar,grid_energy_exported_from_battery,battery_energy_exported,battery_energy_imported_from_grid,battery_energy_imported_from_solar,consumer_energy_imported_from_grid,consumer_energy_imported_from_solar,consumer_energy_imported_from_battery,raw_timestamp,total_battery_charge,total_grid_energy_exported,total_home_usage,total_battery_discharge
0,2020-07-31 10:00:00,0.0,0.0,0.0,0.0,0,20.0,0.0,0.0,0.0,0.0,2020-07-31T10:00:00+02:00,20.0,127.0,NaN,NaN
1,2020-07-31 10:30:00,513.0,0.0,248.0,0.0,0,0.0,100.0,0.0,165.0,0.0,2020-07-31T10:30:00+02:00,100.0,248.0,NaN,NaN
2,2020-07-31 11:00:00,594.0,0.0,360.0,0.0,0,0.0,100.0,0.0,134.0,0.0,2020-07-31T11:00:00+02:00,100.0,360.0,NaN,NaN
3,2020-07-31 11:30:00,1286.0,0.0,359.0,0.0,0,0.0,780.0,0.0,147.0,0.0,2020-07-31T11:30:00+02:00,780.0,359.0,NaN,NaN
4,2020-07-31 12:00:00,2038.0,1.0,0.0,0.0,0,0.0,1050.0,1.0,988.0,0.0,2020-07-31T12:00:00+02:00,1050.0,NaN,NaN,NaN


In [36]:
df=pd.DataFrame()
timestamps = pd.to_datetime(df_raw["timestamp"], dayfirst=False, errors="raise")
df["date"]=timestamps.dt.date
df["time"]=timestamps.dt.time
df["month"]=timestamps.dt.month

In [37]:
df.head()

,date,time,month
0,2020-07-31,10:00:00,7
1,2020-07-31,10:30:00,7
2,2020-07-31,11:00:00,7
3,2020-07-31,11:30:00,7
4,2020-07-31,12:00:00,7


In [38]:
df['solar_energy_exported'] = df_raw['solar_energy_exported']
df['total_home_usage']=df_raw['consumer_energy_imported_from_grid']+df_raw['consumer_energy_imported_from_solar']+df_raw['consumer_energy_imported_from_battery']

In [47]:
# compute solar surplus (at minimum it must be 0)
df['solar_surplus']=df['solar_energy_exported'] - df['total_home_usage']
df.solar_surplus=df.solar_surplus.apply(lambda x: max(0,x))
df.head()

,date,time,month,solar_energy_exported,total_home_usage,solar_surplus
0,2020-07-31,10:00:00,7,0.0,0.0,0.0
1,2020-07-31,10:30:00,7,513.0,165.0,348.0
2,2020-07-31,11:00:00,7,594.0,134.0,460.0
3,2020-07-31,11:30:00,7,1286.0,147.0,1139.0
4,2020-07-31,12:00:00,7,2038.0,989.0,1049.0


In [40]:
df.count()

date                     93944
time                     93944
month                    93944
solar_energy_exported    93944
total_home_usage         93944
solar_surplus            93944
dtype: int64

In [43]:
df[(df.solar_surplus>0)].count()

date                     31227
time                     31227
month                    31227
solar_energy_exported    31227
total_home_usage         31227
solar_surplus            31227
dtype: int64

In [45]:
# check NaN values
df.dropna().count()

date                     93944
time                     93944
month                    93944
solar_energy_exported    93944
total_home_usage         93944
solar_surplus            93944
dtype: int64

In [48]:
df['solar_surplus'].describe()

count    93944.000000
mean       247.711310
std        476.339903
min          0.000000
25%          0.000000
50%          0.000000
75%        250.503569
max       2456.356447
Name: solar_surplus, dtype: float64